In [50]:
import urllib.request
from bs4 import BeautifulSoup
import re
import pandas as pd

In [91]:
#NYC museums
base = "https://www.tripadvisor.com/Attractions-g60763-Activities-c49-New_York_City_New_York.html#ATTRACTION_LIST"
base1 = "http://www.tripadvisor.com"
namelist =[]
addrlist =[]
reviewlist = []
ratinglist =[]
descriptionlist = []
time_feelist = []
lnglist = []
latlist = []
for i in range(3,5):
    a = str(i * 30)
    url1 = "https://www.tripadvisor.com/Attractions-g60763-Activities-c49"
    url2 = "-New_York_City_New_York.html#ATTRACTION_LIST"
    url3 = "-oa" + a
    url = url1 + url3 + url2
    ht = urllib.request.urlopen(url).read()
    soup = BeautifulSoup(ht,'lxml')

#find museum url
    for t in soup.find_all('div',class_ = 'property_title'):
        murl = base1 + t.a.get('href')
        mht = urllib.request.urlopen(murl).read()
        msoup = BeautifulSoup(mht, "lxml")
        
        #find museum name
        for t in msoup.find_all('h1',class_='heading_name'):
            name = t.text.strip()
            namelist.append(name)
            print(name)

        #find address & lat & lng
        for t in msoup.find_all('span',class_='format_address'):
            address = t.text.strip()[9:]
            addrlist.append(address)
            print(address)
        latlng = msoup.find_all('div',class_='mapContainer')
        if latlng:
            for t in latlng:
                lng = t.get('data-lng').strip()
                lat = t.get('data-lat').strip()   
        else:
            lng = -999
            lat = -999
        lnglist.append(lng)
        latlist.append(lat)
        print('lat',lat,'lng',lng)
        
        #find number of reviews
        reviews = msoup.find_all('span',class_='rate sprite-rating_rr rating_rr') 
        if reviews:
            for t in reviews:
                totalreview = t.img.get('content').strip()
                totalreview = int(float(totalreview))
        else:
            totalreview = -999
        reviewlist.append(totalreview)    
        print('reviews:',totalreview)
              
        #find visitor rating 
        
        ratings = msoup.find_all('a',class_ ='more') 
        if ratings:
            for t in ratings:
                rating = t.get('content')
                if rating:
                    print(rating.strip())
        else:
            rating = '-999'
            print('rating:',rating)
        ratinglist.append(rating)
        #find recommendation length of visit and fee
        details = msoup.find_all('div', class_='detail_section details')
        if details:
            dets=[]
            for t in details:
                if t.div:
                    detail = t.div.text.strip().replace('\n'," ")
                    dets.append(detail)
            if len(dets)>1:
                time_fee = dets[0].strip()
                if time_fee[-1]==':':
                        time_fee = time_fee+' <1 hour'
                try:
                    description = dets[-1].split('...')[1].strip()
                    description = re.sub('read more','',description)
                except:
                    description = dets[-1].strip()
                    description = re.sub('read more','',description)
            else:
                if dets[0][0]=='R':
                    time_fee = dets[0].strip()
                    if time_fee[-1]==':':
                        time_fee = time_fee+' <1 hour'
                    description = '-999'
                else:
                    time_fee = '-999'
                    try:
                        description = dets[0].split('...')[1].strip()
                        description = re.sub('read more','',description)
                    except:
                        description = dets[-1].strip()
                        description = re.sub('read more','',description)
        else:
            time_fee = '-999'
            description = '-999'
        time_feelist.append(time_fee)
        descriptionlist.append(description)
        print(time_fee)
        print(description)


Tiles For America
Murly Square, Greenwich Avenue W 11th, New York City, NY
lat 40.73654 lng -74.0011
reviews: 5
8
-999
-999
Lilac Gallery
833 Broadway, Suite 6, New York City, NY 10003-4700
lat 40.73391 lng -73.991196
reviews: 5
4
Recommended length of visit: <1 hour
Lilac Gallery aims to discover brand new art styles and art forms. Our main focus is on emerging international artists that explore new media in their concept with cutting edge techniques, breaking the barrier of conventionality with expressive pieces. Handling a roster of contemporary artists and remain to focus on having unique, desirable, thought provoking and original art works in our gallery as well as showcasing American and European Fine Art, we aim to achieve excellence in every regard, maintaining a multitude of hand-selected works in our collection. Client satisfaction is undoubtedly one of the key pillars upon which the gallery was built, as we strive to meet each individual's customized needs by taking the time

In [93]:
d = {'name':namelist,'address':addrlist,'lat':latlist, 'lng':lnglist, 'rating':ratinglist, 'detail':time_feelist,'total_review':reviewlist,'description':descriptionlist} 
museum = pd.DataFrame(d)
museum.to_csv('museum.csv',encoding='utf-8')

In [94]:
museum

,address,description,detail,lat,lng,name,rating,total_review
0,"Murly Square, Greenwich Avenue W 11th, New Yor...",-999,-999,40.73654,-74.0011,Tiles For America,8,5
1,"833 Broadway, Suite 6, New York City, NY 10003...",Lilac Gallery aims to discover brand new art s...,Recommended length of visit: <1 hour,40.73391,-73.991196,Lilac Gallery,4,5
2,"280 Broome St, New York City, NY 10002-3702",-999,-999,40.71858,-73.99118,Kehila Kedosha Janina,4,5
3,"991 5th Ave, New York City, NY 10028-0101","Founded in 1897 by fifty American Irishmen, th...",-999,40.77781,-73.96309,American Irish Historical Society,8,4
4,"155 Avenue C, New York City, NY 10009-5410",-999,-999,40.725765,-73.97797,Museum of Reclaimed Urban Space (MoRUS),6,4
5,"Intersection of Eighth Avenue, Broadway, and W...",-999,-999,-999,-999,Discovering Columbus,6,5
6,"47 Fifth Ave, Between 11th and 12th streets, N...",-999,-999,40.73425,-73.99445,Salmagundi Museum of American Art,4,5
7,"60 E 8th St Apt 12P, New York City, NY 10003-6539",The Museum of Interesting Things is a travelin...,Recommended length of visit: <1 hour,40.730553,-73.99305,The Museum of Interesting Things,4,5
8,"6 Pennyfield Avenue, New York City, NY 10465",-999,-999,40.805225,-73.79152,Maritime Industry Museum at Fort Schuyler,4,5
9,"104 E 126th St, South side of 129th, near inte...",-999,-999,40.805283,-73.93801,The National Jazz Museum in Harlem,5,4
